Однонаправленная LSTM

Импортируем необходимые зависимости

In [14]:
import numpy as np
import re
import nltk
from nltk.tokenize import word_tokenize
from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding
from keras.preprocessing import sequence
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.sequence import pad_sequences

Загрузим текст для обучения из txt файла

In [15]:
with open('texts/451.txt', 'r', encoding='windows-1251') as file:
    text = file.read()

Создадим токенизатор и словарь слов

In [16]:
tokens = word_tokenize(text)

word_dict = {}
for word in tokens:
    if word not in word_dict:
        word_dict[word] = len(word_dict) + 1

reverse_word_dict = {v: k for k, v in word_dict.items()}
print(reverse_word_dict)

sequences = [word_dict[word] for word in tokens]

{1: '*', 2: '451', 3: '?', 4: 'по', 5: 'Фаренгейту', 6: '[', 7: '1', 8: '-', 9: '233', 10: 'С', 11: '.', 12: '(', 13: 'Здесь', 14: 'и', 15: 'далее', 16: '–', 17: 'прим.', 18: 'перев', 19: ')', 20: ']', 21: 'температура', 22: ',', 23: 'при', 24: 'которой', 25: 'книжные', 26: 'страницы', 27: 'воспламеняются', 28: 'сгорают', 29: 'дотла…', 30: 'Эта', 31: 'книга', 32: 'с', 33: 'благодарностью', 34: 'посвящается', 35: 'Дону', 36: 'Конгдону', 37: 'Если', 38: 'тебе', 39: 'дали', 40: 'разлинованную', 41: 'бумагу', 42: 'пиши', 43: 'по-своему', 44: 'Хуан', 45: 'Рамон', 46: 'Хименес', 47: '2', 48: '1881–1958', 49: 'испанский', 50: 'поэт', 51: 'лауреат', 52: 'Нобелевской', 53: 'премии', 54: '1956', 55: 'Часть', 56: 'первая', 57: 'Домашний', 58: 'очаг', 59: 'саламандра', 60: 'Жечь', 61: 'было', 62: 'удовольствием', 63: 'А', 64: 'особым', 65: 'смотреть', 66: 'как', 67: 'огонь', 68: 'поедает', 69: 'вещи', 70: 'наблюдать', 71: 'они', 72: 'чернеют', 73: 'меняются', 74: 'В', 75: 'кулаках', 76: 'зажат', 7

Создадим обучающие примеры

In [17]:
X = []
y = []
sequence_length = 50

for i in range(len(sequences) - sequence_length):
    X.append(sequences[i:i + sequence_length])
    y.append(sequences[i + sequence_length])

X = np.array(X)
y = to_categorical(y, num_classes=len(word_dict) + 1)

Создадим и обучим модель

In [18]:
embedding_dim = 50
hidden_units = 128

model = Sequential()
model.add(Embedding(input_dim=len(word_dict) + 1, output_dim=embedding_dim, input_length=sequence_length))
model.add(LSTM(hidden_units))
model.add(Dense(len(word_dict) + 1, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [26]:
model.fit(X, y, batch_size=128, epochs=100)

Epoch 1/100
121/121 [==============================] - 13s 110ms/step - loss: 1.6542 - accuracy: 0.6957
Epoch 2/100
121/121 [==============================] - 13s 108ms/step - loss: 1.6013 - accuracy: 0.7039
Epoch 3/100
121/121 [==============================] - 13s 107ms/step - loss: 1.5538 - accuracy: 0.7133
Epoch 4/100
121/121 [==============================] - 13s 106ms/step - loss: 1.5049 - accuracy: 0.7222
Epoch 5/100
121/121 [==============================] - 13s 105ms/step - loss: 1.4571 - accuracy: 0.7324
Epoch 6/100
121/121 [==============================] - 13s 105ms/step - loss: 1.4111 - accuracy: 0.7395
Epoch 7/100
121/121 [==============================] - 13s 106ms/step - loss: 1.3650 - accuracy: 0.7493
Epoch 8/100
121/121 [==============================] - 13s 106ms/step - loss: 1.3258 - accuracy: 0.7570
Epoch 9/100
121/121 [==============================] - 13s 106ms/step - loss: 1.2852 - accuracy: 0.7663
Epoch 10/100
121/121 [==============================] - 13s 106m

Попробуем сгенерировать текст

In [21]:
def generate_text(seed_text):
    seed_tokens = word_tokenize(seed_text)
    seed_sequence = [word_dict[word] for word in seed_tokens]

    for _ in range(50):
        input_sequence = pad_sequences([seed_sequence], maxlen=sequence_length)
        predicted_probs = model.predict(input_sequence)[0]
        predicted_word_index = np.random.choice(np.argsort(predicted_probs)[-5:][::-1])
        seed_sequence.append(predicted_word_index)
        seed_sequence = seed_sequence[-sequence_length:]

    return seed_text + ' ' + ' '.join([reverse_word_dict[index] for index in seed_sequence])

In [29]:
seed_text = 'огонь'
generated_text = generate_text(seed_text)

print(generated_text)

1/1 [==============================] - 0s 21ms/step
огонь того это совершенно стену . В нем люди за только был другой , встала вместо опять удовольствием была был тепло ни и – говорила час , что , представляете ее кивнула : она в порядке . Я стояла же , как птичка в фантастической конуре от его профессиональных дождь ;
